In [ ]:
from sensor_core import SensorManager
from sensor_core import strg_manager
from sensor_core import ser_manager
import numpy as np
from itertools import product

In [ ]:
# Find available serial ports
ser_manager.find_serial()

In [ ]:
# USER DEFINED PARAMS
channel_key = ["red", "infrared", "green", 
                "redc", "greenc", "bluec"]
commport = '/dev/ttyACM0'                    # Name of comm port
baudrate = 115200                            # bits/second
window_size = 1                              # Number of points to acquire before updating plot

In [ ]:
# Custom Serial Acquisition Function
def acquire_data(ser, window_size, num_channel):
    """ serial acquisition function handler. All parameters are internally generated
    Must include the following parameters:
    :param ser: serial port
    :param window_size: number of frames to capture before passing to shared memory object. Default 1
    :param num_channel: number of channels to capture. 
    """
    ser_data = np.zeros((window_size, num_channel))
    channel_data = np.array([])

    for i in product((range(window_size)), range(num_channel)):
        try:
            ser_data[i] = ser.readline().decode().strip()
        except ValueError:
            pass
        # If any zeros
    
    for i in range(window_size):
        if any(ser_data[i, :] == 0):
            pass
        else:
            channel_data = np.append(channel_data, ser_data[i][:])
    return channel_data

In [ ]:
# Test custom serial function
ser = ser_manager.SerialManager(commport=commport, 
                                baudrate=baudrate, 
                                frame_shape=(100, 10, 6))
ser.setup_serial()
ser.acquire_data(func=acquire_data)

In [ ]:
# Instantiate Sensor Manager class
sm = SensorManager(ser_channel_key=ser_channel_key,
                   plot_channel_key=plot_channel_key,
                   commport=commport, 
                   baudrate=baudrate, 
                   dtype=np.float32,
                   data_mode='line',
                   frame_shape = (100, 10, 3), # num_points, window_size, num_channels
                   rotate_seconds=5.0,
                   start_stream_ingest=True)

In [ ]:
# Create 'update process' to handle acquired data
update_process = sm.update_data_process(save_data=True, 
                            filepath='./serial_db.sqlite3',
                            func=acquire_data)

# Create 'plot process' to handle plotting. Returns plot object as well
plot_process, plot = sm.setup_plotting_process()

In [ ]:
# Start update process, start plotting process
## Recommended to use in-built start_process function to avoid Windows OS issues
sm.start_process(update_process)
sm.start_process(plot_process)

In [ ]:
# Show plot
plot.show()